Подключаем необходимые библиотеки


In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import warnings
warnings.filterwarnings('ignore')


Объявляем функции


In [ ]:
#Вычисляет среднюю абсолютную процентную ошибку
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#Вычисляет медианную абсолютную процентную ошибку
def median_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

#Печатает рассчитанные значения коэффициента детерминации, средней и медианной абсолютных ошибок
def print_metrics(prediction, val_y):
    val_mae = mean_absolute_error(val_y, prediction)
    median_AE = median_absolute_error(val_y, prediction)
    r2 = r2_score(val_y, prediction)

    print('')
    print('R\u00b2: {:.2}'.format(r2))
    print('')
    print('Средняя абсолютная ошибка: {:.3} %'.format(mean_absolute_percentage_error(val_y, prediction)))
    print('Медианная абсолютная ошибка: {:.3} %'.format(median_absolute_percentage_error(val_y, prediction)))


Загружаем датасет и делаем первичную обработку


In [ ]:
#При помощи библиотеки pandas считываем csv-файл и преобразуем его в формат датафрейма (таблицы)
file_path = 'https://raw.githubusercontent.com/ValeriaErmolina/Analysis-of-Moscow-apartments/master/moscow_apartments.csv'
df = pd.read_csv(file_path)

#Выводим 5 первых строк датафрейма
df.head(5)

In [ ]:
#Создаем новый столбец Стоимость 1 кв.м путем построчного деления стоимостей квартир на их общие площади
df['price_metr'] = df['price']/df['total_area']

#Выводим сводную информацию о датафрейме и его столбцах (признаках)
df.info()


In [ ]:
#Вычисляем столбцы с категорийными признаками, затем заменяем их на числа
categorical_columns = df.columns[df.dtypes == 'object']
labelencoder = LabelEncoder()
for column in categorical_columns:
    df[column] = labelencoder.fit_transform(df[column])
    print(dict(enumerate(labelencoder.classes_)))

#Выводим сводную информацию о датафрейме и его столбцах (признаках), чтобы убедиться, что теперь они все содержат цифровые значения
df.info()


Создаем целевую переменную, делим датасет на выборки



In [ ]:
#Назначаем целевой переменной цену 1 кв. метра, а можно и цену всей квартиры, тогда будет y = df['price']
y = df['price_metr']

#Создаем список признаков, на основании которых будем строить модели
features = [
            'floors_in_the_house', 
            'floor', 
            'house_type', 
            'number_of_rooms', 
            'total_area',
            'deadline',
            'distance_to_nearest_metro',
            'distance_to_center',
            'distance_to_mkad',
            'distance_to_green_areas',
            'distance_to_sports_complexes',
            'distance_to_industrial_enterprises',
            'distance_to_treatment_facilities'
           ]

#Создаем датафрейм, состоящий из признаков, выбранных ранее
X = df[features]

#Проводим случайное разбиение данных на выборки для обучения (train) и валидации (val), по умолчанию в пропорции 0.75/0.25
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


Модель Random forest


In [ ]:
#Создаем регрессионную модель случайного леса 
rf_model = RandomForestRegressor(n_estimators=2000, 
                                 n_jobs=-1,  
                                 bootstrap=False,
                                 criterion='mse',
                                 max_features=3,
                                 random_state=1,
                                 max_depth=55,
                                 min_samples_split=5
                                 )

#Проводим подгонку модели на обучающей выборке 
rf_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
rf_prediction = rf_model.predict(val_X).round(0)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из валидационной выборки с предсказанными моделью
print_metrics(rf_prediction, val_y)


Модель XGBoost


In [ ]:
#Создаем регрессионную модель XGBoost
xgb_model = xgb.XGBRegressor(objective ='reg:gamma', 
                             learning_rate = 0.01,
                             max_depth = 45, 
                             n_estimators = 2000,
                             nthread = -1,
                             eval_metric = 'gamma-nloglik', 
                             )

#Проводим подгонку модели на обучающей выборке 
xgb_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
xgb_prediction = xgb_model.predict(val_X).round(0)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из валидационной выборки с предсказанными моделью
print_metrics(xgb_prediction, val_y)


Усреднение предсказаний моделей



In [ ]:
#Усредняем предсказания обоих моделей
prediction = rf_prediction * 0.5 + xgb_prediction * 0.5 

#Вычисляем и печатаем величины ошибок для усредненного предсказания
print_metrics(prediction, val_y)


Изучаем важность признаков в модели Random forest


In [ ]:
#Рассчитываем важность признаков в модели Random forest
importances = rf_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

#Печатаем рейтинг признаков
print("Рейтинг важности признаков:")
for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, features[indices[f]], importances[indices[f]]))

#Строим столбчатую диаграмму важности признаков
plt.figure()
plt.title("Важность признаков")
plt.bar(range(X.shape[1]), importances[indices], color="g", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()


In [ ]:
file_flats_path = 'https://raw.githubusercontent.com/ValeriaErmolina/Analysis-of-Moscow-apartments/master/test.csv'
flats = pd.read_csv(file_flats_path)
flats.head(5)
for column in categorical_columns:
    flats[column] = labelencoder.fit_transform(flats[column])
flats_data_for_prediction = flats.drop('price', axis=1)

rf_price_metr = rf_model.predict(flats_data_for_prediction)
xgb_price_metr = xgb_model.predict(flats_data_for_prediction)
price_metr = rf_price_metr * 0.5 + xgb_price_metr * 0.5

flats = flats.drop('floors_in_the_house', axis=1)
flats = flats.drop('floor', axis=1)
flats = flats.drop('house_type', axis=1)
flats = flats.drop('number_of_rooms', axis=1)
flats = flats.drop('deadline', axis=1)
flats = flats.drop('distance_to_nearest_metro', axis=1)
flats = flats.drop('distance_to_center', axis=1)
flats = flats.drop('distance_to_mkad', axis=1)
flats = flats.drop('distance_to_green_areas', axis=1)
flats = flats.drop('distance_to_sports_complexes', axis=1)
flats = flats.drop('distance_to_industrial_enterprises', axis=1)
flats = flats.drop('distance_to_treatment_facilities', axis=1)
flats['price_prediction'] = ((price_metr * flats['total_area']).round(-1)).astype(int)
flats = flats.drop('total_area', axis=1)
flats['mistake'] = np.abs(flats['price_prediction'] - flats['price'])
flats['mistake_percentage'] = flats['mistake'] / flats['price']

flats.head(20)

